# 05 — Robustness Results & Pareto Selection

Loads the latest `runs/robustness_*` summary, plots Sharpe vs MaxDD,
computes a Pareto front, and helps you pick candidate configurations.


In [10]:
from pathlib import Path
import pandas as pd
import numpy as np

# 🔧 Set to your project path (preset for you)
PROJECT_OVERRIDE = r"C:\Users\speed\Desktop\Forex CFD's system"
ROOT = Path(PROJECT_OVERRIDE)

RUNS = ROOT / "runs"

# Manual override (optional): point directly to a specific robustness folder
ROBUSTNESS_DIR_OVERRIDE = None  # e.g. ROOT/"runs/robustness_20250929_1201"

def newest_robustness_dir(base: Path) -> Path | None:
    cands = sorted([p for p in base.glob("robustness_*") if p.is_dir()],
                   key=lambda p: p.stat().st_mtime, reverse=True)
    return cands[0] if cands else None

ROB_DIR = ROBUSTNESS_DIR_OVERRIDE or newest_robustness_dir(RUNS)
assert ROB_DIR is not None, f"No runs/robustness_* folder found under {RUNS}"

print("Using robustness folder:", ROB_DIR)
SUM = ROB_DIR / "summary.csv"
assert SUM.exists(), f"Missing summary.csv in {ROB_DIR}"
df = pd.read_csv(SUM)
print("Rows:", len(df))
df.head(3)


Using robustness folder: C:\Users\speed\Desktop\Forex CFD's system\runs\robustness_quickdemo
Rows: 16


,run,loo,lookbacks,w_tsmom,w_xsec,w_mr,w_volcarry,vc_top_q,vc_bot_q,vc_lookback,target_vol,vol_lookback,max_leverage,cagr,vol,sharpe,maxdd,mar
0,1,-,"(63,126,252)",1.0,0.8,0.6,0.0,0.25,0.25,42,0.1,15,2.0,0.045736,0.025695,1.779986,-0.021725,2.105193
1,2,-,"(63,126,252)",1.0,0.8,0.6,0.0,0.25,0.25,42,0.1,15,3.0,0.039191,0.028408,1.379554,-0.028128,1.393305
2,3,-,"(63,126,252)",1.0,0.8,0.6,0.0,0.25,0.25,42,0.1,30,2.0,0.035552,0.026325,1.350501,-0.021943,1.620177


In [6]:
import pandas as pd

def pareto_front(df, xcol='maxdd', ycol='sharpe', top_n=15):
    dx = df[xcol].abs().astype(float)
    dy = df[ycol].astype(float)
    order = np.lexsort((-dy.values, dx.values))
    mask = [False]*len(df); best_s=-1e9; best_d=1e9
    for i in order:
        d=float(dx.iloc[i]); s=float(dy.iloc[i])
        if d<=best_d and s>=best_s:
            mask[i]=True; best_d=d; best_s=s
    pf = df[mask].copy()
    pf = pf.sort_values([ycol, xcol], ascending=[False, True]).head(top_n)
    return pf

pf = pareto_front(df, xcol='maxdd', ycol='sharpe', top_n=15)

base_cols = ['run','loo','lookbacks','w_tsmom','w_xsec','w_mr']
maybe_cols = ['w_volcarry','vc_top_q','vc_bot_q','vc_lookback']
tail_cols = ['target_vol','vol_lookback','max_leverage','cagr','vol','sharpe','maxdd','mar']

# only keep columns that actually exist
existing = set(df.columns)
cols_show = [c for c in base_cols + maybe_cols + tail_cols if c in existing]

pf_display = pf[cols_show].copy()
pf_display



,run,loo,lookbacks,w_tsmom,w_xsec,w_mr,w_volcarry,target_vol,vol_lookback,max_leverage,cagr,vol,sharpe,maxdd,mar
5,6,-,"(63, 126, 252)",1.0,0.8,0.6,0.0,0.1,30,3.0,0.178493,0.11936,1.495421,-0.025824,6.911919


In [7]:
import plotly.express as px

dsp = df.copy()
dsp['drawdown_abs'] = dsp['maxdd'].abs()

hover_fixed = ['run','lookbacks','w_tsmom','w_xsec','w_mr']
hover_maybe = ['w_volcarry','vc_top_q','vc_bot_q','vc_lookback']
hover_tail  = ['target_vol','vol_lookback','max_leverage','cagr','vol','maxdd','mar']

hover_cols = [c for c in hover_fixed + hover_maybe + hover_tail if c in dsp.columns]

fig = px.scatter(
    dsp, x='drawdown_abs', y='sharpe',
    hover_data=hover_cols,
    title='Sharpe vs |MaxDD| (lower is better on X, higher is better on Y)'
)
fig.update_layout(xaxis_title='|Max Drawdown|', yaxis_title='Sharpe')
fig.show()

topN = df.sort_values('sharpe', ascending=False).head(15)
px.bar(topN, x='run', y='sharpe', title='Top-15 by Sharpe', hover_data=[c for c in topN.columns]).show()



In [8]:
# ✏️ Adjust filters here to shortlist candidates
flt = df.copy()
# Examples:
# if 'w_volcarry' in flt.columns:
#     flt = flt.query("w_volcarry >= 0.3")
# flt = flt.query("maxdd > -0.10")
# flt = flt.query("sharpe >= 1.0")
# flt = flt.query("vol_lookback == 20 and target_vol == 0.12")

short = flt.sort_values(['sharpe','maxdd'], ascending=[False, True]).head(10)

base_cols = ['run','loo','lookbacks','w_tsmom','w_xsec','w_mr']
maybe_cols = ['w_volcarry','vc_top_q','vc_bot_q','vc_lookback']
tail_cols = ['target_vol','vol_lookback','max_leverage','cagr','vol','sharpe','maxdd','mar']
cols_show = [c for c in base_cols + maybe_cols + tail_cols if c in short.columns]

short_display = short[cols_show].copy()
short_display


,run,loo,lookbacks,w_tsmom,w_xsec,w_mr,w_volcarry,target_vol,vol_lookback,max_leverage,cagr,vol,sharpe,maxdd,mar
14,15,-,"(63, 126, 252)",1.0,0.8,0.6,0.0,0.15,20,2.0,0.164424,0.092185,1.783627,-0.033123,4.964062
12,13,-,"(63, 126, 252)",1.0,0.8,0.6,0.0,0.15,15,2.0,0.160919,0.091471,1.759240,-0.032932,4.886397
16,17,-,"(63, 126, 252)",1.0,0.8,0.6,0.0,0.15,30,2.0,0.159920,0.092188,1.734709,-0.032178,4.969914
8,9,-,"(63, 126, 252)",1.0,0.8,0.6,0.0,0.12,20,2.0,0.150381,0.089897,1.672808,-0.030711,4.896638
10,11,-,"(63, 126, 252)",1.0,0.8,0.6,0.0,0.12,30,2.0,0.149394,0.091111,1.639689,-0.030167,4.952219
15,16,-,"(63, 126, 252)",1.0,0.8,0.6,0.0,0.15,20,3.0,0.211462,0.129776,1.629434,-0.039277,5.383819
6,7,-,"(63, 126, 252)",1.0,0.8,0.6,0.0,0.12,15,2.0,0.143086,0.089403,1.600455,-0.032457,4.408458
17,18,-,"(63, 126, 252)",1.0,0.8,0.6,0.0,0.15,30,3.0,0.209525,0.130987,1.599587,-0.038668,5.418621
2,3,-,"(63, 126, 252)",1.0,0.8,0.6,0.0,0.10,20,2.0,0.140144,0.087807,1.596037,-0.026199,5.349287
4,5,-,"(63, 126, 252)",1.0,0.8,0.6,0.0,0.10,30,2.0,0.139172,0.088683,1.569319,-0.025832,5.387594


In [9]:
# Pick one run id to use in the next step (override.yaml location)
CHOSEN_RUN_ID = int(short['run'].iloc[0])  # or set manually, e.g. 7

OVR = ROB_DIR / f"run_{CHOSEN_RUN_ID:04d}" / "override.yaml"
assert OVR.exists(), f"override.yaml not found for run {CHOSEN_RUN_ID} at {OVR}"
print("Chosen override:", OVR)

# Save a small pointer file for other notebooks/scripts
POINTER = ROB_DIR / "chosen_override_path.txt"
POINTER.write_text(str(OVR), encoding="utf-8")
print("Saved pointer:", POINTER)


Chosen override: C:\Users\speed\Desktop\Forex CFD's system\runs\robustness_with_volcarry\run_0015\override.yaml
Saved pointer: C:\Users\speed\Desktop\Forex CFD's system\runs\robustness_with_volcarry\chosen_override_path.txt
